In [20]:
import pandas as pd
from fredapi import Fred
import os
from openpyxl import load_workbook
from typing import List, Tuple, Union
from pathlib import Path

In [5]:
api="958ccd9c67808caf9f941367daf6e812"
fred = Fred(api_key=api)

In [10]:
Monthly_Tickers=[
    "SAHMREALTIME",
    "RECPROUSM156N",
    "PCEDGC96",
    "PCESC96",
    "RSAFS",
    "DSPIC96",
    "PSAVERT",
    "TOTALSA",
    "REVOLSL",
    "NONREVSL",
    "DGORDER",
    "INDPRO",
    "TCU",
    "HOUST",
    "PERMIT",
    "HSN1F",
    "EXHOSLUSM495S",
    "PAYEMS",
    "ADPMNUSNERSA",
    "UNRATE",
    "U6RATE",
    "CIVPART",
    "EMRATIO",
    "JTSJOR",
    "JTSHIR",
    "JTSTSR",
    "CPIAUCSL",
    "CPILFESL",
    "PPIFIS",
    "PCEPI",
    "PCEPILFE",
    "UMCSENT",
    "T5YIFR",
    "AHETPI",
    "CSUSHPINSA",
    "DTWEXBGS",
    "IQ",
    "IR"
]

Quarterly_Tickers=[
    "GDPC1",
    "NGDPSAXDCUSQ",
    "GDPNOW",
    "PCECC96",
    "PCNDGC96",
    "PNFIC1",
    "PRFIC1",
    "GCE",
    "ECIWAG",
    "EXPGS",
    "IMPGS",
    "NETEXP"
]

Weekly_Tickers=[
    "ICSA",
    "CCSA",
    "MORTGAGE30US"
]

Daily_Tickers = [
    "T10YIE",
    "DTWEXBGS",
    "FEDFUNDS",
    "DGS2",
    "DGS5",
    "DGS10",
    "DBAA"
]

In [11]:
def Fetch(ticker_list, fred, freq):
    values_dfs = []
    dates_dfs = []

    # Set Frequency
    f = str(freq).strip().upper()
    if f in ("MONTHLY", "M"):
        period_code = "M"
    elif f in ("QUARTERLY", "Q"):
        period_code = "Q"
    elif f in ("WEEKLY", "W"):
        period_code = "W"
    elif f in ("ANNUAL", "A", "Y"):
        period_code = "A"
    elif f in ("DAILY", "D"):
        period_code = "D"
    else:
        raise ValueError(f"Unsupported frequency: {freq}")

    for ticker in ticker_list:
        try:
            s = fred.get_series(ticker)
        except Exception as e:
            print(f"Warning: couldn't fetch {ticker}: {e}")
            continue

        s.index = pd.to_datetime(s.index)
        df = s.reset_index()
        df.columns = ["release_date", ticker]

        if period_code == "D":
            df["Time"] = df["release_date"].dt.floor("D")
        else:
            df["Time"] = df["release_date"].dt.to_period(period_code).dt.to_timestamp()

        grouped = df.groupby("Time").agg({ticker: "last", "release_date": "last"})
        values_dfs.append(grouped[[ticker]])
        dates_dfs.append(grouped[["release_date"]].rename(columns={"release_date": ticker}))

    if not values_dfs:
        return pd.DataFrame(), pd.DataFrame()

    Values = pd.concat(values_dfs, axis=1, sort=True)
    Dates = pd.concat(dates_dfs, axis=1, sort=True)
    Values.index.name = "Time"
    Dates.index.name = "Time"

    return Values, Dates

Monthly_Values,Monthly_Dates = Fetch(Monthly_Tickers,fred,"M")
Quarterly_Values,Quarterly_Dates = Fetch(Quarterly_Tickers,fred,"Q")
Weekly_Values,Weekly_Dates = Fetch(Weekly_Tickers,fred,"W")
Daily_Values,Daily_Dates = Fetch(Daily_Tickers,fred,"D")

In [12]:
Monthly_Values

,SAHMREALTIME,RECPROUSM156N,PCEDGC96,PCESC96,RSAFS,DSPIC96,PSAVERT,TOTALSA,REVOLSL,NONREVSL,...,PPIFIS,PCEPI,PCEPILFE,UMCSENT,T5YIFR,AHETPI,CSUSHPINSA,DTWEXBGS,IQ,IR
Time,,,,,,,,,,,,,,,,,,,,,
1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-01,0.17,0.34,2113.9,10863.2,723033.0,18036.2,5.0,16.042,1300321.06,3742429.93,...,148.303,126.743,126.121,60.7,2.27,31.26,331.671,119.8269,152.4,140.7
2025-07-01,0.10,0.64,2152.2,10888.2,727414.0,18077.3,4.8,16.896,1311491.90,3749312.19,...,149.341,126.949,126.418,61.7,2.32,31.34,331.127,122.1088,152.8,141.0
2025-08-01,0.13,0.96,2171.4,10910.9,732010.0,18097.2,4.6,16.802,1305533.90,3755633.42,...,149.160,127.285,126.705,58.2,2.35,31.46,NaN,120.6028,153.2,141.4


In [22]:
# to csv
# find project root relative to this script
# BASE_DIR = Path(__file__).resolve().parent.parent Use this when in a python project instead of this notebook
BASE_DIR = Path.cwd()
RAW_LOCATION = BASE_DIR / "MacroDashboard" / "Raw Data"

# create directory if it doesn't exist
RAW_LOCATION.mkdir(parents=True, exist_ok=True)

# save files
Monthly_Values.to_csv(RAW_LOCATION / "Monthly_Values.csv")
Monthly_Dates.to_csv(RAW_LOCATION / "Monthly_Dates.csv")
Quarterly_Values.to_csv(RAW_LOCATION / "Quarterly_Values.csv")
Quarterly_Dates.to_csv(RAW_LOCATION / "Quarterly_Dates.csv")
Weekly_Values.to_csv(RAW_LOCATION / "Weekly_Values.csv")
Weekly_Dates.to_csv(RAW_LOCATION / "Weekly_Dates.csv")
Daily_Values.to_csv(RAW_LOCATION / "Daily_Values.csv")
Daily_Dates.to_csv(RAW_LOCATION / "Daily_Dates.csv")

In [17]:
def GetData(Ticker: str, Freq: str, n_lags: int = 4):
    if Freq in ("MONTHLY", "M"):
        period_code = "Monthly"
    elif Freq in ("QUARTERLY", "Q"):
        period_code = "Quarterly"
    elif Freq in ("WEEKLY", "W"):
        period_code = "Weekly"
    elif Freq in ("ANNUAL", "A", "Y"):
        period_code = "Annual"
    elif Freq in ("DAILY", "D"):
        period_code = "Daily"
    else:
        raise ValueError(f"Unsupported frequency: {Freq}")

    base = os.path.join("MacroDashboard", "Raw Data")
    Dates_Path = os.path.join(base, f"{period_code}_Dates.csv")
    Values_Path = os.path.join(base, f"{period_code}_Values.csv")

    Dates = pd.read_csv(Dates_Path)
    Values = pd.read_csv(Values_Path)

    if Ticker not in Values.columns:
        raise KeyError(f"Ticker '{Ticker}' not found in values file.")

    Last_Idx = Values[Ticker].last_valid_index()
    Last_Pos = Values.index.get_loc(Last_Idx)

    Start_Pos = max(0, Last_Pos - n_lags)
    Latest_Values = Values[Ticker].iloc[Start_Pos: Last_Pos + 1].tolist()
    Latest_Date= Dates[Ticker].iloc[Last_Pos]
    return Latest_Date, Latest_Values

In [26]:
# --- Constants ---
INPUT_XLSX = 'C:\\Users\\hp\\Desktop\\Duke Research Opportunity\\Prof. Aguilar Macro Dashboard\\MacroDashBoard\\MacroDashboard\\Economic Dashboard V1-Template.xlsx'
OUTPUT_XLSX = 'C:\\Users\\hp\\Desktop\\Duke Research Opportunity\\Prof. Aguilar Macro Dashboard\\MacroDashBoard\\MacroDashboard\\Economic Dashboard V1-Template-UPDATE.xlsx'

# --- Helper ---
def _to_python_datetime(dt: Union[pd.Timestamp, str, None]):
    """
    Convert pandas.Timestamp -> python datetime for openpyxl.
    If dt is None or unparseable, return it unchanged.
    """
    if isinstance(dt, pd.Timestamp):
        return dt.to_pydatetime()
    return dt

# --- Load workbook & sheet ---
wb = load_workbook(INPUT_XLSX)
ws = wb.active

# --- Row bounds ---
start_row = 3
max_row = ws.max_row

def write_panel_for_row(
    row: int,
    ticker_col: str,
    freq_col: str,
    date_col: str,
    present_col: str,
    lag_cols: List[str],
    n_lags: int = 4,
) -> None:
    """
    Write one row for a panel (left or right).
    - ticker_col, freq_col, date_col, present_col: column letters (e.g. "B", "E", "C", "F")
    - lag_cols: list of lag column letters in order [Lag1_col, Lag2_col, ...]
    - n_lags: number of lags expected (defaults to 4)
    """
    # read ticker and freq values from the worksheet
    raw_ticker = ws[f"{ticker_col}{row}"].value
    raw_freq = ws[f"{freq_col}{row}"].value

    # If ticker cell is empty -> skip row
    if raw_ticker is None or str(raw_ticker).strip() == "":
        return

    # normalize strings safely
    ticker = str(raw_ticker).strip()
    freq = str(raw_freq).strip() if raw_freq is not None else "M"

    # get data using your GetData function (assumed defined/imported)
    try:
        latest_date, recent_values = GetData(ticker, freq, n_lags=n_lags)
    except Exception as exc:
        # write error to date cell and skip writing values for this row
        ws[f"{date_col}{row}"].value = f"ERR: {exc}"
        return

    # recent_values expected chronological oldest ... most recent
    values = list(recent_values)

    # Ensure fixed length = n_lags + 1 by padding at front (older side) with None
    expected_len = n_lags + 1
    if len(values) < expected_len:
        pad_len = expected_len - len(values)
        values = [None] * pad_len + values

    # Present is last element, lag1 is second-last, etc.
    present_value = values[-1]
    lag_values = []
    for i in range(1, len(lag_cols) + 1):
        # i=1 -> lag1 => values[-2], general: values[-1 - i]
        idx = -1 - i
        # safe access: if index out of range, return None
        try:
            lag_values.append(values[idx])
        except IndexError:
            lag_values.append(None)

    # write date (convert pandas Timestamp -> python datetime for openpyxl)
    ws[f"{date_col}{row}"].value = _to_python_datetime(latest_date)

    # write present and lag values
    ws[f"{present_col}{row}"].value = present_value
    for col_letter, lag_value in zip(lag_cols, lag_values):
        ws[f"{col_letter}{row}"].value = lag_value


# --- Panel definitions (columns) ---
# Left panel: B = ticker, E = freq, C = Latest Date, F = Present, G,H,I,J = Lag1..Lag4
left_panel = {
    "ticker_col": "B",
    "freq_col": "E",
    "date_col": "C",
    "present_col": "F",
    "lag_cols": ["G", "H", "I", "J"],
}

# Right panel: M = ticker, P = freq, N = Latest Date, Q = Present, R,S,T,U = Lag1..Lag4
right_panel = {
    "ticker_col": "M",
    "freq_col": "P",
    "date_col": "N",
    "present_col": "Q",
    "lag_cols": ["R", "S", "T", "U"],
}

# --- Iterate rows and write panels ---
for r in range(start_row, max_row + 1):
    write_panel_for_row(r, **left_panel)
    write_panel_for_row(r, **right_panel)

# --- Save updated workbook ---
wb.save(OUTPUT_XLSX)
print(f"Updated workbook saved to: {OUTPUT_XLSX}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hp\\Desktop\\Duke Research Opportunity\\Prof. Aguilar Macro Dashboard\\MacroDashBoard\\MacroDashboard\\Economic Dashboard V1-Template.xlsx'